In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats as st
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

In [2]:
def cohen_glass_hedges(x,y):
    nx = len(x)
    ny = len(y)
    pstdnmr = (nx-1)*x.var(ddof=1) + (ny-1)*y.var(ddof=1)   # (n-1)(var1+var2)
    pstddnm = nx + ny - 2                                   # (n-1)*2
    pstd = (pstdnmr / pstddnm)**(0.5)
    cohen = (x.mean() - y.mean()) / pstd
    glass = (x.mean() - y.mean()) / x.std(ddof=1)
    hedges = cohen * (1 - 3/(4 * (nx+ny) - 9) )             # cohen * (1-3/(8*n-9))
    return (cohen, glass, hedges)

In [3]:
a = pd.read_csv('aeroscan.csv')

#### Dataset parameter description

1. RF – Respiratory frequency (breaths/minute)
2. HF – Heart rate (beats/minute)
3. RER – The ratio of exhaled CO2 to inspired O2 (respiratory exchange rate)
4. Fat - kcal/h fat consumed
5. Carb - kcal/h carbohydrates consumed
6. Energy – kcal/h total energy consumed

* 1, 2, 3 - effort level
* a, b - before/after treatment

In [6]:
a.shape

(39, 37)

In [9]:
a.columns

Index(['Name', 'RF1a', 'RF1b', 'RF2a', 'RF2b', 'RF3a', 'RF3b', 'HF1a', 'HF1b',
       'HF2a', 'HF2b', 'HF3a', 'HF3b', 'RER1a', 'RER1b', 'RER2a', 'RER2b',
       'RER3a', 'RER3b', 'Fat1a', 'Fat1b', 'Fat2a', 'Fat2b', 'Fat3a', 'Fat3b',
       'Carb1a', 'Carb1b', 'Carb2a', 'Carb2b', 'Carb3a', 'Carb3b', 'Energy1a',
       'Energy1b', 'Energy2a', 'Energy2b', 'Energy3a', 'Energy3b'],
      dtype='object')